In [15]:
import os
import random
from PIL import Image
from torchvision import transforms
import torch
from torch.utils.data import Dataset, DataLoader
import shutil
from tqdm import tqdm

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [16]:
root_folder = "../Resized_MyDataset"
categories = ["Anger", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

In [17]:
def select_images_for_processing(root_folder, categories, augment_ratio=0.3, grayscale_ratio=0.2):
    augmentation_selection = {}
    grayscale_selection = {}
    
    for category in categories:
        folder_path = os.path.join(root_folder, category)
        images = [img for img in os.listdir(folder_path) if img.lower().endswith(('png', 'jpg', 'jpeg'))]
        random.shuffle(images)
        
        total_images = len(images)
        augment_count = int(total_images * augment_ratio)
        grayscale_count = int(total_images * grayscale_ratio)
        
        augmentation_selection[category] = images[:augment_count]
        grayscale_selection[category] = images[augment_count:augment_count + grayscale_count]
    
    return augmentation_selection, grayscale_selection

augmentation_selection, grayscale_selection = select_images_for_processing(root_folder, categories)

In [18]:
augmentation_transforms = transforms.Compose([
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1),
    transforms.RandomRotation(degrees=15),
    transforms.RandomHorizontalFlip()
])

def apply_augmentation(root_folder, augmentation_selection, categories):
    for category, images in augmentation_selection.items():
        folder_path = os.path.join(root_folder, category)
        for img_name in tqdm(images, desc=f"Augmenting {category}"):
            img_path = os.path.join(folder_path, img_name)
            img = Image.open(img_path).convert("RGB")
            augmented_img = augmentation_transforms(img)
            augmented_img.save(img_path)  # Replace original image

apply_augmentation(root_folder, augmentation_selection, categories)

Augmenting Surprise: 100%|██████████| 3971/3971 [00:41<00:00, 96.28it/s] 


In [19]:
def apply_grayscale(root_folder, grayscale_selection, categories):
    grayscale_transform = transforms.Grayscale(num_output_channels=3)
    
    for category, images in grayscale_selection.items():
        folder_path = os.path.join(root_folder, category)
        for img_name in tqdm(images, desc=f"Grayscaling {category}"):
            img_path = os.path.join(folder_path, img_name)
            img = Image.open(img_path).convert("RGB")
            grayscale_img = grayscale_transform(img)
            grayscale_img.save(img_path)  # Replace original image

apply_grayscale(root_folder, grayscale_selection, categories)

Grayscaling Surprise: 100%|██████████| 2647/2647 [00:19<00:00, 134.70it/s]
